In [2]:
import pandas as pd 

In [27]:
df=pd.read_excel('dubizzle.xlsx')

In [35]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 361205 entries, 0 to 982621
Data columns (total 17 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   Type               361205 non-null  object        
 1   Ownership          361205 non-null  object        
 2   Area (m²)          361205 non-null  float64       
 3   Bedrooms           361205 non-null  int32         
 4   Bathrooms          361205 non-null  int32         
 5   Furnished          361205 non-null  object        
 6   Payment Option     355768 non-null  object        
 7   Completion status  361205 non-null  object        
 8   Delivery Term      361205 non-null  object        
 9   Listing date       361205 non-null  datetime64[ns]
 10  Price              361205 non-null  float64       
 11  Down Payment       361205 non-null  float64       
 12  Amenities          361205 non-null  object        
 13  Level              361205 non-null  int32   

In [5]:
import mysql.connector

In [7]:
# Connect to MySQL server 
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password=""
)
cursor = conn.cursor()



In [8]:
# Create a new database
cursor.execute("CREATE DATABASE IF NOT EXISTS dubizzle;")
cursor.execute("USE dubizzle;")


In [9]:
# Step 2: Load CSV files into DataFrames
fact_table = pd.read_csv('Property_Facts.csv')
location_dim = pd.read_csv('Location_Dim.csv')
type_dim = pd.read_csv('Type_Dim.csv')
ownership_dim = pd.read_csv('Ownership_Dim.csv')
payment_option_dim = pd.read_csv('Payment_Option_Dim.csv')
price_type_dim = pd.read_csv('Price_Type_Dim.csv')
amenities_dim = pd.read_csv('Amenities_Dim.csv')


In [56]:
# Amenities Dimension Table (with IDs)
amenities_dim_columns = [ 'Elevator', 'Maids_Room', 
                         'Built_in_Kitchen_Appliances', 'Pets_Allowed', 
                         'Natural_Gas', 'Pool', 'Private_Garden', 
                         'Covered_Parking', 'Electricity_Meter', 
                         'Balcony', 'Central_AC_heating', 'Water_Meter', 
                         'Landline', 'Security']
amenities_dim = pd.DataFrame(columns=amenities_dim_columns)



pf_columns = [ 'Location_ID','Type_ID', 'Ownership_ID', 'Payment_Option_ID', 
                            'Price_Type_ID', 'Amenities_ID', 'Area_M2', 'Bedrooms', 'Bathrooms', 'Price', 
                            'Down_Payment', 'Level', 'Listing_Date', 'Total_Amenities', 'Fully_Equipped' ]
amenities_dim = pd.DataFrame(columns=pf_columns)


location_dim.rename(columns={
    'Location.1': 'Location'
}, inplace=True)

In [57]:

#SQL queries to create the tables
create_amenities_dim = """
CREATE TABLE IF NOT EXISTS Amenities_Dim (
    Amenities_ID INT PRIMARY KEY,
    Elevator TINYINT,
    Maids_Room TINYINT,
    Built_in_Kitchen_Appliances TINYINT,
    Pets_Allowed TINYINT,
    Natural_Gas TINYINT,
    Pool TINYINT,
    Private_Garden TINYINT,
    Covered_Parking TINYINT,
    Electricity_Meter TINYINT,
    Balcony TINYINT,
    Central_AC_heating TINYINT,
    Water_Meter TINYINT,
    Landline TINYINT,
    Security TINYINT
);
"""

create_location_dim = """
CREATE TABLE IF NOT EXISTS Location_Dim (
    Location_ID INT PRIMARY KEY,
    City VARCHAR(255),
    Location_1 VARCHAR(255)
);
"""

create_ownership_dim = """
CREATE TABLE IF NOT EXISTS Ownership_Dim (
    Ownership_ID INT PRIMARY KEY,
    Ownership VARCHAR(255)
);
"""

create_payment_option_dim = """
CREATE TABLE IF NOT EXISTS Payment_Option_Dim (
    Payment_Option_ID INT PRIMARY KEY,
    Payment_Option VARCHAR(255)
);
"""

create_type_dim = """
CREATE TABLE IF NOT EXISTS Type_Dim (
    Type_ID INT PRIMARY KEY,
    Type VARCHAR(255),
    Furnished TINYINT,
    Completion_Status VARCHAR(255),
    Delivery_Term VARCHAR(255)
);
"""

create_price_type_dim = """
CREATE TABLE IF NOT EXISTS Price_Type_Dim (
    Price_Type_ID INT PRIMARY KEY,
    Price_Type VARCHAR(255)
);
"""

create_property_facts = """
CREATE TABLE IF NOT EXISTS Property_Facts (
    Property_ID INT PRIMARY KEY,
    Location_ID INT,
    Type_ID INT,
    Ownership_ID INT,
    Payment_Option_ID INT,
    Price_Type_ID INT,
    Amenities_ID INT,
    Area_M2 DECIMAL(10,2),
    Bedrooms INT,
    Bathrooms INT,
    Price DECIMAL(15,2),
    Down_Payment DECIMAL(15,2),
    Level INT,
    Listing_Date DATETIME,
    Total_Amenities INT,
    Fully_Equipped TINYINT,
    FOREIGN KEY (Location_ID) REFERENCES Location_Dim(Location_ID),
    FOREIGN KEY (Type_ID) REFERENCES Type_Dim(Type_ID),
    FOREIGN KEY (Ownership_ID) REFERENCES Ownership_Dim(Ownership_ID),
    FOREIGN KEY (Payment_Option_ID) REFERENCES Payment_Option_Dim(Payment_Option_ID),
    FOREIGN KEY (Amenities_ID) REFERENCES Amenities_Dim(Amenities_ID)
);
"""

# Step 4: Execute the create table queries
cursor.execute(create_amenities_dim)
cursor.execute(create_location_dim)
cursor.execute(create_ownership_dim)
cursor.execute(create_payment_option_dim)
cursor.execute(create_type_dim)
cursor.execute(create_price_type_dim)
cursor.execute(create_property_facts)


print("Tables created successfully in the 'dubblze.")


Tables created successfully in the 'dubblze.


In [24]:
# Step 4: Alter tables to make ID columns auto-increment
alter_location_dim = """
ALTER TABLE Location_Dim 
MODIFY Location_ID INT AUTO_INCREMENT;
"""

alter_ownership_dim = """
ALTER TABLE Ownership_Dim 
MODIFY Ownership_ID INT AUTO_INCREMENT;
"""

alter_payment_option_dim = """
ALTER TABLE Payment_Option_Dim 
MODIFY Payment_Option_ID INT AUTO_INCREMENT;
"""

alter_type_dim = """
ALTER TABLE Type_Dim 
MODIFY Type_ID INT AUTO_INCREMENT;
"""

alter_price_type_dim = """
ALTER TABLE Price_Type_Dim 
MODIFY Price_Type_ID INT AUTO_INCREMENT;
"""

alter_amenities_dim = """
ALTER TABLE Amenities_Dim 
MODIFY Amenities_ID INT AUTO_INCREMENT;
"""


# Execute the ALTER TABLE queries
cursor.execute(alter_location_dim)
cursor.execute(alter_ownership_dim)
cursor.execute(alter_payment_option_dim)
cursor.execute(alter_type_dim)
cursor.execute(alter_price_type_dim)


# Commit the changes
conn.commit()


try _last

In [23]:
print("Location Dim Columns:", location_dim.columns.tolist())
print("Type Dim Columns:", type_dim.columns.tolist())
print("Ownership Dim Columns:", ownership_dim.columns.tolist())
print("Payment Option Dim Columns:", payment_option_dim.columns.tolist())
print("Price Type Dim Columns:", price_type_dim.columns.tolist())
print("Amenities Dim Columns:", amenities_dim.columns.tolist())


Location Dim Columns: ['City', 'Location.1', 'Location_ID']
Type Dim Columns: ['Type', 'Furnished', 'Completion status', 'Delivery Term', 'Type_ID']
Ownership Dim Columns: ['Ownership', 'Ownership_ID']
Payment Option Dim Columns: ['Payment Option', 'Payment_Option_ID']
Price Type Dim Columns: ['Price Type', 'Price_Type_ID']
Amenities Dim Columns: ['Elevator', 'Maids Room', 'Built in Kitchen Appliances', 'Pets Allowed', 'Natural Gas', 'Pool', 'Private Garden', 'Covered Parking', 'Electricity Meter', 'Balcony', 'Central A/C & heating', 'Water Meter', 'Landline', 'Security', 'Amenities_ID']


In [58]:
# Updated Step 4: Define insert queries for each table (without IDs)
insert_amenities_query = """
INSERT INTO Amenities_Dim (Elevator, Maids_Room, Built_in_Kitchen_Appliances, Pets_Allowed, 
                           Natural_Gas, Pool, Private_Garden, Covered_Parking, Electricity_Meter, 
                           Balcony, Central_AC_heating, Water_Meter, Landline, Security) 
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

insert_location_query = """
INSERT INTO Location_Dim (City, Location_1) 
VALUES (%s, %s)
"""

insert_ownership_query = """
INSERT INTO Ownership_Dim (Ownership) 
VALUES (%s)
"""

insert_payment_option_query = """
INSERT INTO Payment_Option_Dim (Payment_Option) 
VALUES (%s)
"""

insert_type_query = """
INSERT INTO Type_Dim (Type, Furnished, Completion_Status, Delivery_Term) 
VALUES (%s, %s, %s, %s)
"""

insert_price_type_query = """
INSERT INTO Price_Type_Dim (Price_Type) 
VALUES (%s)
"""




In [11]:
# Function to insert data into the database
def insert_data(data, query):
    for _, row in data.iterrows():
        cursor.execute(query, tuple(row))
        conn.commit()



In [63]:
insert_data(amenities_dim, insert_amenities_query)

In [65]:
insert_data(ownership_dim, insert_ownership_query)
insert_data(payment_option_dim, insert_payment_option_query)
insert_data(type_dim, insert_type_query)
print("Data inserted successfully into all tables.")

Data inserted successfully into all tables.


In [121]:
alter_property_facts = """
ALTER TABLE Property_Facts 
MODIFY Property_ID INT AUTO_INCREMENT;
"""
cursor.execute(alter_property_facts)

In [1]:
# Step 3: Define your insert query without Property_ID
insert_property_facts_query = """
INSERT INTO Property_Facts (Location_ID, Type_ID, Ownership_ID, Payment_Option_ID, 
                            Price_Type_ID, Amenities_ID, Area_M2, Bedrooms, Bathrooms, Price, 
                            Down_Payment, Level, Listing_Date, Total_Amenities, Fully_Equipped) 
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# Step 5: Insert data into Property_Facts
insert_data(fact_table, insert_property_facts_query)


In [90]:
print()

In [91]:
cursor.execute("SET FOREIGN_KEY_CHECKS = 0;")


# Step 3: Define your insert query
insert_property_facts_query = """
INSERT INTO Property_Facts (Property_ID,Location_ID, Type_ID, Ownership_ID, Payment_Option_ID, 
                            Price_Type_ID, Amenities_ID, Area_M2, Bedrooms, Bathrooms, Price, 
                            Down_Payment, Level, Listing_Date, Total_Amenities, Fully_Equipped) 
VALUES (%s,%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# Step 4: Function to insert data into the database
def insert_data(data, query):
    for index, row in data.iterrows():    
        cursor.execute(query, tuple(row))

# Step 5: Insert data into Property_Facts
insert_data(fact_table, insert_property_facts_query)

# Step 6: Re-enable foreign key checks
cursor.execute("SET FOREIGN_KEY_CHECKS = 1;")

**1. What is the average price of properties in each city?**


In [ ]:
query = """
SELECT L.City, AVG(P.Price) AS Average_Price
FROM Property_Facts P
JOIN Location_Dim L ON P.Location_ID = L.Location_ID
GROUP BY L.City;
"""
df = pd.read_sql(query, conn)
print(df)


**2. How many properties have a pool and are furnished in each city?**

In [ ]:
query = """
SELECT L.City, COUNT(*) AS Total_Properties
FROM Property_Facts P
JOIN Amenities_Dim A ON P.Amenities_ID = A.Amenities_ID
JOIN Location_Dim L ON P.Location_ID = L.Location_ID
WHERE A.Pool = 'Yes' AND A.Furnished = 'Yes'
GROUP BY L.City;
"""
df = pd.read_sql(query, conn)
print(df)

**3. How many properties are available for each payment option?**

In [ ]:
query = """
SELECT PO.Payment_Option, COUNT(P.Property_ID) AS Property_Count
FROM Property_Facts P
JOIN Payment_Option_Dim PO ON P.Payment_Option_ID = PO.Payment_Option_ID
GROUP BY PO.Payment_Option;
"""
df = pd.read_sql(query, conn)
print(df)


**4. What is the highest-priced property in each city?**


In [ ]:
query = """
SELECT L.City, MAX(P.Price) AS Highest_Price
FROM Property_Facts P
JOIN Location_Dim L ON P.Location_ID = L.Location_ID
GROUP BY L.City;
"""
df = pd.read_sql(query, conn)
print(df)



**5. Which type of property has the most listings?**

In [ ]:
query = """
SELECT T.Type, COUNT(P.Property_ID) AS Listing_Count
FROM Property_Facts P
JOIN Type_Dim T ON P.Type_ID = T.Type_ID
GROUP BY T.Type
ORDER BY Listing_Count DESC;
"""
df = pd.read_sql(query, conn)
print(df)

**6. What is the total number of amenities for properties listed in each location?**


In [ ]:
query = """
SELECT L.Location_1, SUM(P.Total_Amenities) AS Total_Amenities
FROM Property_Facts P
JOIN Location_Dim L ON P.Location_ID = L.Location_ID
GROUP BY L.Location_1;
"""
df = pd.read_sql(query, conn)
print(df)


**7. How many properties have a garden and a balcony?**


In [ ]:
query = """
SELECT COUNT(*) AS Property_Count
FROM Property_Facts P
JOIN Amenities_Dim A ON P.Amenities_ID = A.Amenities_ID
WHERE A.Garden = 'Yes' AND A.Balcony = 'Yes';
"""
df = pd.read_sql(query, conn)
print(df)


**8. What is the minimum property price in each city?**


In [ ]:
query = """
SELECT L.City, MIN(P.Price) AS Min_Property_Price
FROM Property_Facts P
JOIN Location_Dim L ON P.Location_ID = L.Location_ID
GROUP BY L.City;
"""
df = pd.read_sql(query, conn)
print(df)


**9. What is the average down payment required for properties in each location?**


In [ ]:
query = """
SELECT L.Location_1, AVG(P.Down_Payment) AS Average_Down_Payment
FROM Property_Facts P
JOIN Location_Dim L ON P.Location_ID = L.Location_ID
GROUP BY L.Location_1;
"""
df = pd.read_sql(query, conn)
print(df)


**10. How many properties are listed by each ownership type?**


In [ ]:
query = """
SELECT O.Ownership, COUNT(P.Property_ID) AS Property_Count
FROM Property_Facts P
JOIN Ownership_Dim O ON P.Ownership_ID = O.Ownership_ID
GROUP BY O.Ownership;
"""
df = pd.read_sql(query, conn)
print(df)


**11. What is the total price of all properties listed in each city?**

In [ ]:
query = """
SELECT L.City, SUM(P.Price) AS Total_Price
FROM Property_Facts P
JOIN Location_Dim L ON P.Location_ID = L.Location_ID
GROUP BY L.City;
"""
df = pd.read_sql(query, conn)
print(df)

**12. What is the highest price of properties in each type category?**


In [ ]:
query = """
SELECT T.Type, MAX(P.Price) AS Highest_Price
FROM Property_Facts P
JOIN Type_Dim T ON P.Type_ID = T.Type_ID
GROUP BY T.Type;
"""
df = pd.read_sql(query, conn)
print(df)


**13. Find all properties that are fully equipped and have a natural gas connection.**


In [ ]:
query = """
SELECT P.*
FROM Property_Facts P
JOIN Amenities_Dim A ON P.Amenities_ID = A.Amenities_ID
WHERE A.Fully_Equipped = 'Yes' AND A.Natural_Gas = 'Yes';
"""
df = pd.read_sql(query, conn)
print(df)


**14. How many properties are listed each month?**


In [ ]:
query = """
SELECT DATE_FORMAT(P.Listing_Date, '%Y-%m') AS Month, COUNT(P.Property_ID) AS Property_Count
FROM Property_Facts P
GROUP BY Month;
"""
df = pd.read_sql(query, conn)
print(df)